# Third Parser: Selecting a Passage
by Niek Veldhuis
UC Berkeley

July 2017 2017

Main difference between this parser and the `Second_JSON_parser.ipynb` is the ability to define a region within a tablet to parse by giving a start label and/or a stop label. This may be used in order to omit colophons or to select one exercise among several on a school tablet.

This requires some extra parsing of the list of text IDs where labels are now optional. The list of text IDs may look like this:

- dcclt/Q000039
- dcclt/P247864 - r i 16
- dcclt/P228683 r 1 - r 4
- dcclt/P230849 r 1 -

The first will parse the entirety of `Q000039`; the second will take everything of `P247864` up to and including `r i 16`. The third will only take `r 1` to (and including) `r 4` of `P228683`. The fourth will parse everything of `P230849` starting from `r 1`. The format of the labels depends on the structure of the text object and the way it has been edited. The labels must follow exactly the form they have in the online [ORACC](http://oracc.org) edition.


## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [1]:
import pandas as pd   
import requests
import zipfile
import tqdm
import numpy as np

In [2]:
name = input('Filename: ')

Filename: test.txt


In [3]:
with open('text_ids/' + name, 'r') as f:
    pqxnos = f.read().splitlines()
pqxnos = [no.strip() for no in pqxnos]             # strip spaces left and right
pqxnos = [x for x in pqxnos if not x == ""] # strip empty lines
nos_labels = [no.split(' ', 1) if " " in no else [no, '-'] for no in pqxnos] # isolate text ID
for label in nos_labels:    # separate start label and stop label
    label[1] = label[1].split('-')

## 1.5 Parse JSON files
The `parsejson()` below includes `id_word` which has the form `TextID.LineID.WordID` - in other words, line and text ID can be derived from it. The parser also cptures data on broken lines etcetera.

Parsejson() takes a single argument, an [ORACC](http://oracc.org) text in JSON format. The parameters `startlabel`, `stoplabel`, and the variables `keep` and `label` are stored in the dictionary `parameters` outside of the function. While iterating through the JSON object the function changes the variables `keep` and `label`. 

The variable `keep` is initially set to `False` if there is a `startlabel`. When `startlabel` equals `label` (or if there is no `startlabel` to begin with) the parser starts to collect lemmatizations. The parser stops when `label` equals `endlabel`. 

The list `dollar_keys` (also outside of the function) stores the relevant field names when capturing line breaks etc.

In [4]:
def parsejson(text, parameters):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject, parameters)
        if "label" in JSONobject:
            parameters["label"] = JSONobject["label"]
        if parameters["label"] == parameters["startlabel"]:
            parameters["keep"] = True
        if parameters["label"] == parameters["endlabel"]:
            parameters["keep"] = False
        if parameters["keep"] == True or parameters["label"] == parameters["endlabel"]: # the "or" statement ensures that the line
            if "f" in JSONobject:             # corresponding to the endlabel is included.
                lemma = JSONobject["f"]
                lemma["id_word"] = JSONobject["ref"]
                lemma["label"] = parameters["label"]
                lemma["id_text"] = parameters["id_text"]
                lemm_l.append(lemma)
            if "strict" in JSONobject and JSONobject["strict"] == "1":
                lemma = {key: JSONobject[key] for key in dollar_keys}
                lemma["id_word"] = JSONobject["ref"] + ".0"
                lemma["id_text"] = parameters["id_text"]
                lemm_l.append(lemma)
    return

##  1.6 Call the Parser for each Textid

In [5]:
lemm_l = []
dollar_keys = ["extent", "scope", "state"]
for pqx in tqdm.tqdm(nos_labels):
    project = pqx[0][:-8].lower()
    textid = pqx[0][-7:].upper()
    parameters = {"startlabel":pqx[1][0].strip(), "endlabel":pqx[1][1].strip(), "label":None, 
                  "keep": False, "id_text": project + "/" + textid}
    if parameters["startlabel"] == "":
        parameters["keep"] = True
    url = "http://oracc.museum.upenn.edu/" + project + "/corpusjson/" + textid + ".json"  
    r = requests.get(url).json()
    try:
        parsejson(r, parameters)
    except:
        print(url + ' is not available or not complete')

100%|██████████| 2/2 [00:03<00:00,  1.43s/it]


## 1.7 Transform the Data into a DataFrame
The word_l list is transformed into a Pandas dataframe for further manipulation.

For various reasons not all JSON files will have all data types that potentially exist in an [ORACC](http://oracc.org) signature. Only Sumerian words have a `base`, so if your data set has no Sumerian, this column will not exist in the DataFrame.  If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. Since such fields are referenced in the code below (sections 2-4) the next cell will check for the existence of each column and create an empty column if necessary.

In [6]:
words = pd.DataFrame(lemm_l)
fields = ['base', 'cf', 'cont', 'epos', 'extent', 'form', 'gw', 'id_word',
          'label', 'lang', 'morph', 'norm', 'norm0', 'pos', 'scope', 'sense']
for field in fields:
    if not field in words.columns:
        words[field] = ''
words = words.fillna('') # replace Missing Values by empty string
words.head(100)

,base,cf,cont,delim,epos,extent,form,gdl,gw,id_text,id_word,label,lang,morph,norm,norm0,pos,scope,sense,state
0,,,,,,,{d}utu-gin₇,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",,blms/P282597,P282597.2.1,o 1,sux,,,,,,,
1,,,,,,,e₃-ta,"[{'v': 'e₃', 'id': 'P282597.2.2.0', 'break': '...",,blms/P282597,P282597.2.2,o 1,sux,,,,,,,
2,,,,,,,uru₂-zu,"[{'v': 'uru₂', 'id': 'P282597.2.3.0', 'break':...",,blms/P282597,P282597.2.3,o 1,sux,,,,,,,
3,,,,,,,e-NE,"[{'v': 'e', 'id': 'P282597.2.4.0', 'break': 'd...",,blms/P282597,P282597.2.4,o 1,sux,,,,,,,
4,,,,,,,x,"[{'x': 'ellipsis', 'id': 'P282597.2.5.0', 'bre...",,blms/P282597,P282597.2.5,o 1,sux,,,,,,,
5,,kīma,,,PRP,,ki-ma,"[{'v': 'ki', 'id': 'P282597.3.1.0', 'break': '...",like,blms/P282597,P282597.3.1,o 1a,akk-x-stdbab,,kīma,,PRP,,like,
6,,Šamaš,,,DN,,{d}ša₂-maš,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",1,blms/P282597,P282597.3.2,o 1a,akk-x-stdbab,,Šamaš,,DN,,1,
7,,waṣû,,,V,,ṣa-am-ma,"[{'v': 'ṣa', 'id': 'P282597.3.3.0', 'delim': '...",go out,blms/P282597,P282597.3.3,o 1a,akk-x-stdbab,,ṣâmma,,V,,go out,
8,,ālu,,,N,,IRI-ka,"[{'s': 'IRI', 'id': 'P282597.3.4.0', 'role': '...",city,blms/P282597,P282597.3.4,o 1a,akk-x-stdbab,,ālka,,N,,city,
9,,hiāṭu,,,V,,hi-i-ṭi,"[{'v': 'hi', 'id': 'P282597.3.5.0', 'delim': '...",supervise,blms/P282597,P282597.3.5,o 1a,akk-x-stdbab,,hīṭi,,V,,check,


## 1.8 Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [7]:
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* frag (transliteration; including flags)
* gdl_utf8 (cuneiform)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* sig (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# 3. Manipulate for Analysis on Line level (e.g. phylogenetics)
For analyses that use a line as unit of analysis (e.g. lines in lexical texts as analyzed in the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project) one may need to create lemmas and combine these into lines by using the `id_line` variable.

## 3.1 Create Lemmas and Adjust Bases
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string.

For Sumerian projects each lemmatized word has a `base` (the word without morphology). For non-lemmatized words a place-holder base is created that consists of the transliteration of the word. If you are not working with Sumerian data.

In [8]:
words["lemma"] = words.apply(lambda r: (r["cf"] + '[' + r["gw"] + ']' + r["pos"]) 
                            if r["cf"] != '' else r['form'] + '[NA]NA', axis=1)
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = words.apply(lambda r: r["base"] if r["base"] != '' or r['label'] == '' else r['form'], axis=1)

## 3.2 Group by Line
In the `words` dataframe each word has a separate row. In order to change this into a line-by-line representation we use the Pandas `.groupby` function, using the `id_line`, `id_text` and `label` fields as arguments. `label` is a human-readable line number in the format `o ii 3`: obverse column 2, line 3. The field `id_line` is an integer that is created from `id_word`, which has the format `IDText.IDLine.IDWord`, for instance `P296528.23.1`. 

The fields that are aggregated are `lemma`, `base`, `extent`, and `scope`. The fields `extent` and `scope` represent data on the number of broken lines. If you work with Akkadian data you want to leave out the field `base`.

In [9]:
#words['id_line'] = [int(wordid[wordid.find('.')+1:wordid.rfind('.')]) for wordid in words['id_word']]
words['id_line'] = [int(wordid.split('.')[1]) for wordid in words['id_word']]

In [10]:
lines = words.groupby([words['id_text'], words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join,
        'state': ''.join    
    }).reset_index()
lines        

,id_text,id_line,label,lemma,base,extent,scope,state
0,blms/P274259,2,o 1,x-x-šu₂[NA]NA x[NA]NA isqu[share]N x[NA]NA x[N...,x-x-šu₂ x i-siq x x x x,,,
1,blms/P274259,3,o 2,x[NA]NA ummiānu[craftsman]N kisallu[court]N bī...,x um-ma-ni ki-sal E₂ ṭup-pi x KISAL E₂ ṭup-pi x,,,
2,blms/P274259,4,o 3,x[NA]NA dug[speak]V/t x-tuku-tuku-x[NA]NA x-ia...,x dug₄ x-tuku-tuku-x x-ia ga-na lu-uq-bi-kum₂-...,,,
3,blms/P274259,5,o 4,x[NA]NA edubbaʾa[scribal-school]N til[live]V/i...,x e₂-dub-ba-a til₃ x me₂-eṭ-lu-ti-ka i-na E₂ ṭ...,,,
4,blms/P274259,6,o 5,x[NA]NA ahāzu[take]V ittu[sign]N ul[not]MOD ed...,x ta-hu-zu i-da-as-su ul ti-i-de ŋiškim nu-i₃-x,,,
5,blms/P282597,2,o 1,{d}utu-gin₇[NA]NA e₃-ta[NA]NA uru₂-zu[NA]NA e-...,{d}utu-gin₇ e₃-ta uru₂-zu e-NE x,,,
6,blms/P282597,3,o 1a,kīma[like]PRP Šamaš[1]DN waṣû[go-out]V ālu[cit...,ki-ma {d}ša₂-maš ṣa-am-ma IRI-ka hi-i-ṭi,,,
7,blms/P282597,4,,,,1,line,ruling
8,blms/P282597,5,o 2,am[NA]NA u-mu-un[NA]NA x-ga[NA]NA {d}utu-gin₇[...,am u-mu-un x-ga {d}utu-gin₇ e₃-ta uru₂ x,,,
9,blms/P282597,6,o 2a,rīmu[wild-bull]N bēlu[lord]N nišu[people]N kīm...,ri-mu be-el ni-ši ki-ma {d}ša₂-maš ṣa-am-ma IR...,,,


Note that the new `id_line` field is not a line number in the traditional sense of the word (this is `label`) but a number used to organize lines in the appropriate order.

## 3.3 Save in CSV Format

In [11]:
filename = name[:-4]
with open('output/' + filename + '.csv', 'w') as w:
    lines.to_csv(w, encoding='utf8', index=False)